# 1. PSO Meta-Heuristic

In [31]:
### EMS Filter

######1. Constantes
# length in x-axis; width in y-axis; height in z-axis
class RotationType:
    RT_LWH = 0
    RT_HLW = 1
    RT_HWL = 2
    RT_WHL = 3
    RT_WLH = 4
    RT_LHW = 5
    
    ALL = [RT_LWH, RT_HLW, RT_HWL, RT_WHL, RT_WLH, RT_LHW]


# (x, y, z) --> (length, width, height)
class Axis:
    LENGTH = 0
    WIDTH = 1
    HEIGHT = 2
    
    ALL = [LENGTH, WIDTH, HEIGHT]
    
##### 2.Metodos Auxiliares

from decimal import Decimal

DEFAULT_NUMBER_OF_DECIMALS = 3
START_POSITION = [0, 0, 0]

def get_limit_number_of_decimals(number_of_decimals):
    return Decimal('1.{}'.format('0' * number_of_decimals))

def set_to_decimal(value, number_of_decimals):
    number_of_decimals = get_limit_number_of_decimals(number_of_decimals)

    return Decimal(value).quantize(number_of_decimals)

def rect_intersect(item1, item2, x, y):
    """Estimate whether two items get intersection in one dimension.
    Args:
        item1, item2: any two items in item list.
        x,y: Axis.LENGTH/ Axis.Height/ Axis.WIDTH.
    Returns:
        Boolean variable: False when two items get intersection in one dimension; True when two items do not intersect in one dimension.
    """
    
    d1 = item1.get_dimension() 
    d2 = item2.get_dimension() 
    
    cx1 = item1.position[x] + d1[x]/2 
    cy1 = item1.position[y] + d1[y]/2
    cx2 = item2.position[x] + d2[x]/2 
    cy2 = item2.position[y] + d2[y]/2
    
    ix = max(cx1, cx2) - min(cx1, cx2) # ix: |cx1-cx2|
    iy = max(cy1, cy2) - min(cy1, cy2) # iy: |cy1-cy2|
    
    return ix < (d1[x] + d2[x])/2 and iy < (d1[y] + d2[y])/2

def intersect(item1, item2):
    """Estimate whether two items get intersection in 3D dimension.
    Args:
        item1, item2: any two items in item list.
    Returns:
        Boolean variable: False when two items get intersection; True when two items do not intersect.
    """
    
    return ( 
    rect_intersect(item1, item2, Axis.LENGTH, Axis.HEIGHT) and # xz dimension
    rect_intersect(item1, item2, Axis.HEIGHT, Axis.WIDTH) and # yz dimension
    rect_intersect(item1, item2, Axis.LENGTH, Axis.WIDTH)) # xy dimension


def stack(item1, item2):
    """Stack two items with same length, width, height or any two of three sides are same.
    Args:
        item1, item2: any two items in item list.
    Return:
        item1/ stacked_item_list/ stacked_item.
    """
    
    if (
        item1.length == item2.length and
        item1.width == item2.width and
        item1.height == item2.height
    ):
        stacked_item_1 = Item(item1.name + item2.name, item1.length + item2.length, 
                              item1.width, item1.height, item1.weight + item2.weight) #(2l, w, h)
        stacked_item_2 = Item(item1.name + item2.name, item1.length, item1.width + item2.width, 
                              item1.height, item1.weight + item2.weight) #(l, 2w, h)
        stacked_item_3 = Item(item1.name + item2.name, item1.length, item1.width, 
                              item1.height + item2.height, item1.weight + item2.weight) #(l, w, 2h)
        
        stacked_item_list = [stacked_item_1, stacked_item_2, stacked_item_3]
        
        return stacked_item_list
        
    elif ( 
        item1.length == item2.length and
        item1.width == item2.width and
        item1.height != item2.height
    ):
        stacked_item = Item(item1.name + item2.name, item1.length, item1.width, 
                            item1.height + item2.height, item1.weight + item2.weight) #(l, w, 2h)
        
        return stacked_item
    
    elif (
        item1.length == item2.length and 
        item1.height == item2.height and
        item1.width != item2.width
    ):
        stacked_item = Item(item1.name + item2.name, item1.length, item1.width + item2.width, 
                            item1.height, item1.weight + item2.weight) #(l, 2w, h)
        
        return stacked_item
    
    elif (
        item1.width == item2.width and
        item1.height == item2.height and
        item1.length != item2.length
    ):
        stacked_item = Item(item1.name + item2.name, item1.length + item2.length, 
                            item1.width, item1.height, item1.weight + item2.weight)
        
        return stacked_item #(2l, w, h)
    
    else:
        return item1

#### 3. Contenedor(Bin)

class Bin:
    def __init__(self, size, length, width, height, capacity):
        self.size = size 
        self.length = length
        self.width = width
        self.height = height
        self.capacity = capacity
        self.total_items = 0 # number of total items in one bin
        self.items = [] # item in one bin -- a blank list initially
        self.unplaced_items = []
        self.unfitted_items = [] # unfitted item in one bin -- a blank list initially
        self.number_of_decimals = DEFAULT_NUMBER_OF_DECIMALS
    
    def format_numbers(self, number_of_decimals):
        self.length = set_to_decimal(self.length, number_of_decimals)
        self.height = set_to_decimal(self.height, number_of_decimals)
        self.width = set_to_decimal(self.width, number_of_decimals)
        self.capacity = set_to_decimal(self.capacity, number_of_decimals)
        self.number_of_decimals = number_of_decimals
    
    def get_volume(self):
        return set_to_decimal(
            self.length * self.height * self.width, self.number_of_decimals)
     
    def get_total_weight(self):
        total_weight = 0
        
        for item in self.items:
            total_weight += item.weight
        
        return set_to_decimal(total_weight, self.number_of_decimals)
    
    def get_filling_ratio(self):
        total_filling_volume = 0
        total_filling_ratio = 0
        
        for item in self.items:
            total_filling_volume += item.get_volume()
            
        total_filling_ratio = total_filling_volume / self.get_volume()
        return set_to_decimal(total_filling_ratio, self.number_of_decimals)
    
    def can_hold_item_with_rotation(self, item, pivot): 
        """Evaluate whether one item can be placed into bin with all optional orientations.
        Args:
            item: any item in item list.
            pivot: an (x, y, z) coordinate, the back-lower-left corner of the item will be placed at the pivot.
        Returns:
            a list containing all optional orientations. If not, return an empty list.
        """
        
        fit = False 
        valid_item_position = [0, 0, 0]
        item.position = pivot 
        rotation_type_list = [] 
        
        
        for i in range(0, len(RotationType.ALL)): 
            item.rotation_type = i 
            dimension = item.get_dimension() 
            if (
                pivot[0] + dimension[0] <= self.length and  # x-axis
                pivot[1] + dimension[1] <= self.width and  # y-axis
                pivot[2] + dimension[2] <= self.height     # z-axis
            ):
            
                fit = True
                
                for current_item_in_bin in self.items: 
                    if intersect(current_item_in_bin, item): 
                        fit = False
                        item.position = [0, 0, 0]
                        break 
                
                if fit: 
                    if self.get_total_weight() + item.weight > self.capacity: # estimate whether bin reaches its capacity
                        fit = False
                        item.position = [0, 0, 0]
                        continue 
                    
                    else: 
                        rotation_type_list.append(item.rotation_type) 
            
            else:
                continue 
        
        return rotation_type_list 

    def put_item(self, item, pivot, distance_3d): 
        """Evaluate whether an item can be placed into a certain bin with which orientation. If yes, perform put action.
        Args:
            item: any item in item list.
            pivot: an (x, y, z) coordinate, the back-lower-left corner of the item will be placed at the pivot.
            distance_3d: a 3D parameter to determine which orientation should be chosen.
        Returns:
            Boolean variable: False when an item couldn't be placed into the bin; True when an item could be placed and perform put action.
        """
        
        fit = False 
        rotation_type_list = self.can_hold_item_with_rotation(item, pivot)
        margins_3d_list = []
        margins_3d_list_temp = []
        margin_3d = []
        margin_2d = []
        margin_1d = []
        
        n = 0
        m = 0
        p = 0
        
        if not rotation_type_list:
            return fit 
        
        else:
            fit = True 
            
            rotation_type_number = len(rotation_type_list)
            
            if rotation_type_number == 1: 
                item.rotation_type = rotation_type_list[0] 
                self.items.append(item)
                self.total_items += 1
                return fit 
            
            else: 
                for rotation in rotation_type_list: 
                    item.rotation_type = rotation
                    dimension = item.get_dimension()
                    margins_3d = [distance_3d[0] - dimension[0], 
                                 distance_3d[1] - dimension[1], 
                                 distance_3d[2] - dimension[2]]
                    margins_3d_temp = sorted(margins_3d)
                    margins_3d_list.append(margins_3d)
                    margins_3d_list_temp.append(margins_3d_temp)
                
                while p < len(margins_3d_list_temp):
                    margin_3d.append(margins_3d_list_temp[p][0])
                    p += 1
                
                p = 0
                while p < len(margins_3d_list_temp):
                    if margins_3d_list_temp[p][0] == min(margin_3d):
                        n += 1
                        margin_2d.append(margins_3d_list_temp[p][1])
                    
                    p += 1
                
                if n == 1:
                    p = 0
                    while p < len(margins_3d_list_temp):
                        if margins_3d_list_temp[p][0] == min(margin_3d):
                            item.rotation_type = rotation_type_list[p]
                            self.items.append(item)
                            self.total_items += 1
                            return fit 
                        
                        p += 1
                
                else:
                    p = 0
                    while p < len(margins_3d_list_temp):
                        if (
                            margins_3d_list_temp[p][0] == min(margin_3d) and
                            margins_3d_list_temp[p][1] == min(margin_2d)
                        ):
                            m += 1
                            margin_1d.append(margins_3d_list_temp[p][2])
                        
                        p += 1
                
                if m == 1:
                    p = 0
                    while p < len(margins_3d_list_temp):
                        if (
                            margins_3d_list_temp[p][0] == min(margin_3d) and
                            margins_3d_list_temp[p][1] == min(margin_2d)
                        ):
                            item.rotation_type = rotation_type_list[p]
                            self.items.append(item)
                            self.total_items += 1
                            return fit 
                        
                        p += 1
                
                else:
                    p = 0
                    while p < len(margins_3d_list_temp):
                        if (
                            margins_3d_list_temp[p][0] == min(margin_3d) and
                            margins_3d_list_temp[p][1] == min(margin_2d) and
                            margins_3d_list_temp[p][2] == min(margin_1d)
                        ):
                            item.rotation_type = rotation_type_list[p]
                            self.items.append(item)
                            self.total_items += 1
                            return fit 
                        
                        p += 1
        
    def string(self):
        return "%s(%sx%sx%s, max_weight:%s) vol(%s) item_number(%s) filling_ratio(%s)" % (
            self.size, self.length, self.width, self.height, self.capacity,
            self.get_volume(), self.total_items, self.get_filling_ratio())

##### 4. Boxes(Item)

class Item:
    def __init__(self, name, length, width, height, weight):
        self.name = name
        self.length = length
        self.width = width
        self.height = height
        self.weight = weight
        self.rotation_type = 0 # initial rotation type: (x, y, z) --> (l, w, h)
        self.position = START_POSITION # initial position: (0, 0, 0)
        self.number_of_decimals = DEFAULT_NUMBER_OF_DECIMALS
    
    def format_numbers(self, number_of_decimals):
        self.length = set_to_decimal(self.length, number_of_decimals)
        self.height = set_to_decimal(self.height, number_of_decimals)
        self.width = set_to_decimal(self.width, number_of_decimals)
        self.weight = set_to_decimal(self.weight, number_of_decimals)
        self.number_of_decimals = number_of_decimals
        
    def get_volume(self):
        return set_to_decimal(
            self.length * self.height * self.width, self.number_of_decimals)
    
    def get_dimension(self): # 6 orientation types -- (x-axis, y-axis, z-axis)
        if self.rotation_type == RotationType.RT_LWH:
            dimension = [self.length, self.width, self.height]
        elif self.rotation_type == RotationType.RT_HLW:
            dimension = [self.height, self.length, self.width]
        elif self.rotation_type == RotationType.RT_HWL:
            dimension = [self.height, self.width, self.length]
        elif self.rotation_type == RotationType.RT_WHL:
            dimension = [self.width, self.height, self.length]
        elif self.rotation_type == RotationType.RT_WLH:
            dimension = [self.width, self.length, self.height]
        elif self.rotation_type == RotationType.RT_LHW:
            dimension = [self.length, self.height, self.width]
        else:
            dimension = []
        
        return dimension
        
    def string(self):
        return "%s(%sx%sx%s, weight: %s) pos(%s) rt(%s) vol(%s)" % (
            self.name, self.length, self.width, self.height, self.weight,
            self.position, self.rotation_type, self.get_volume()
        )

#### 5.Empaquetador(Packer)

class Packer:
    def __init__(self):
        self.bins = [] 
        self.unplaced_items = []
        self.placed_items = []
        self.unfit_items = []
        self.total_items = 0
        self.total_used_bins = 0 # not used yet
        self.used_bins = [] # not used yet
    
    def add_bin(self, bin):
        return self.bins.append(bin)
    
    def add_item(self, item): 
        """Add unplaced items into bin's unplaced_items list.
        Args:
            item: an unplaced item.
        Returns:
            The unplaced item is added into bin's unplaced_items list."""
        
        self.total_items += 1
        return self.unplaced_items.append(item) 
    
    def pivot_dict(self, bin, item):
        """For each item to be placed into a certain bin, obtain a corresponding comparison parameter of each optional pivot that the item can be placed.
        Args:
            bin: a bin in bin list that a certain item will be placed into.
            item: an unplaced item in item list.
        Returns:
            a pivot_dict contain all optional pivot point and their comparison parameter of the item.
            a empty dict may be returned if the item couldn't be placed into the bin.
        """
        
        pivot_dict = {}
        can_put = False
        
        for axis in range(0, 3): 
            items_in_bin = bin.items 
            items_in_bin_temp = items_in_bin[:] 
            
            n = 0
            while n < len(items_in_bin):
                pivot = [0, 0, 0] 
                
                if axis == Axis.LENGTH: # axis = 0/ x-axis
                    ib = items_in_bin[n]
                    pivot = [ib.position[0] + ib.get_dimension()[0],
                            ib.position[1],
                            ib.position[2]]
                    try_put_item = bin.can_hold_item_with_rotation(item, pivot) 
                    
                    if try_put_item: 
                        can_put = True
                        q = 0
                        q = 0
                        ib_neigh_x_axis = []
                        ib_neigh_y_axis = []
                        ib_neigh_z_axis = []
                        right_neighbor = False
                        front_neighbor = False
                        above_neighbor = False
                        
                        while q < len(items_in_bin_temp):
                            if items_in_bin_temp[q] == items_in_bin[n]: 
                                q += 1 
                            
                            else:
                                ib_neighbor = items_in_bin_temp[q]
                                
                                if (
                                    ib_neighbor.position[0] > ib.position[0] + ib.get_dimension()[0] and 
                                    ib_neighbor.position[1] + ib_neighbor.get_dimension()[1] > ib.position[1] and 
                                    ib_neighbor.position[2] + ib_neighbor.get_dimension()[2] > ib.position[2] 
                                ): 
                                    right_neighbor = True
                                    x_distance = ib_neighbor.position[0] - (ib.position[0] + ib.get_dimension()[0])
                                    ib_neigh_x_axis.append(x_distance)
                                    
                                elif (
                                    ib_neighbor.position[1] >= ib.position[1] + ib.get_dimension()[1] and 
                                    ib_neighbor.position[0] + ib_neighbor.get_dimension()[0] > ib.position[0] + ib.get_dimension()[0] and 
                                    ib_neighbor.position[2] + ib_neighbor.get_dimension()[2] > ib.position[2] 
                                ):
                                    front_neighbor = True
                                    y_distance = ib_neighbor.position[1] - ib.position[1]
                                    ib_neigh_y_axis.append(y_distance)
                                
                                elif (
                                    ib_neighbor.position[2] >= ib.position[2] + ib.get_dimension()[2] and 
                                    ib_neighbor.position[0] + ib_neighbor.get_dimension()[0] > ib.position[0] + ib.get_dimension()[0] and 
                                    ib_neighbor.position[1] + ib_neighbor.get_dimension()[1] > ib.position[1] 
                                ):
                                    above_neighbor = True
                                    z_distance = ib_neighbor.position[2] - ib.position[2]
                                    ib_neigh_z_axis.append(z_distance)
                                
                                q += 1 
                                
                        if not right_neighbor: 
                            x_distance = bin.length - (ib.position[0] + ib.get_dimension()[0])
                            ib_neigh_x_axis.append(x_distance)
                        
                        if not front_neighbor: 
                            y_distance = bin.width - ib.position[1]
                            ib_neigh_y_axis.append(y_distance)
                        
                        if not above_neighbor: 
                            z_distance = bin.height - ib.position[2]
                            ib_neigh_z_axis.append(z_distance)
                        
                        distance_3D = [min(ib_neigh_x_axis), min(ib_neigh_y_axis), min(ib_neigh_z_axis)]
                        pivot_dict[tuple(pivot)] = distance_3D
                
                elif axis == Axis.WIDTH: # axis = 1/ y-axis
                    ib = items_in_bin[n]
                    pivot = [ib.position[0],
                            ib.position[1] + ib.get_dimension()[1],
                            ib.position[2]]
                    try_put_item = bin.can_hold_item_with_rotation(item, pivot) 
                    
                    if try_put_item: 
                        can_put = True
                        q = 0
                        ib_neigh_x_axis = []
                        ib_neigh_y_axis = []
                        ib_neigh_z_axis = []
                        right_neighbor = False
                        front_neighbor = False
                        above_neighbor = False
                        
                        while q < len(items_in_bin_temp):
                            if items_in_bin_temp[q] == items_in_bin[n]: 
                                q += 1 
                            
                            else:
                                ib_neighbor = items_in_bin_temp[q]
                                
                                if (
                                    ib_neighbor.position[0] >= ib.position[0] + ib.get_dimension()[0] and 
                                    ib_neighbor.position[1] + ib_neighbor.get_dimension()[1] > ib.position[1] + ib.get_dimension()[1] and 
                                    ib_neighbor.position[2] + ib_neighbor.get_dimension()[2] > ib.position[2] 
                                ):
                                    right_neighbor = True
                                    x_distance = ib_neighbor.position[0] - ib.position[0]
                                    ib_neigh_x_axis.append(x_distance)
                                
                                elif (
                                    ib_neighbor.position[1] > ib.position[1] + ib.get_dimension()[1] and 
                                    ib_neighbor.position[0] + ib_neighbor.get_dimension()[0] > ib.position[0] and 
                                    ib_neighbor.position[2] + ib_neighbor.get_dimension()[2] > ib.position[2] 
                                ):
                                    front_neighbor = True
                                    y_distance = ib_neighbor.position[1] - (ib.position[1] + ib.get_dimension()[1])
                                    ib_neigh_y_axis.append(y_distance)
                                
                                elif (
                                    ib_neighbor.position[2] >= ib.position[2] + ib.get_dimension()[2] and 
                                    ib_neighbor.position[0] + ib_neighbor.get_dimension()[0] > ib.position[0] and 
                                    ib_neighbor.position[1] + ib_neighbor.get_dimension()[1] > ib.position[1] + ib.get_dimension()[1] 
                                ):
                                    above_neighbor = True
                                    z_distance = ib_neighbor.position[2] - ib.position[2]
                                    ib_neigh_z_axis.append(z_distance)
                                
                                q += 1
                        
                        if not right_neighbor: 
                            x_distance = bin.length - ib.position[0]
                            ib_neigh_x_axis.append(x_distance)
                        
                        if not front_neighbor: 
                            y_distance = bin.width - (ib.position[1] + ib.get_dimension()[1])
                            ib_neigh_y_axis.append(y_distance)
                        
                        if not above_neighbor: 
                            z_distance = bin.height - ib.position[2]
                            ib_neigh_z_axis.append(z_distance)
                        
                        distance_3D = [min(ib_neigh_x_axis), min(ib_neigh_y_axis), min(ib_neigh_z_axis)]
                        pivot_dict[tuple(pivot)] = distance_3D
            
                elif axis == Axis.HEIGHT: # axis = 2/ z-axis
                    ib = items_in_bin[n]
                    pivot = [ib.position[0],
                            ib.position[1],
                            ib.position[2] + ib.get_dimension()[2]]
                    try_put_item = bin.can_hold_item_with_rotation(item, pivot) 
                    
                    if try_put_item: 
                        can_put = True
                        q = 0
                        ib_neigh_x_axis = []
                        ib_neigh_y_axis = []
                        ib_neigh_z_axis = []
                        right_neighbor = False
                        front_neighbor = False
                        above_neighbor = False
                        
                        while q < len(items_in_bin_temp):
                            if items_in_bin_temp[q] == items_in_bin[n]: 
                                q += 1 
                            
                            else:
                                ib_neighbor = items_in_bin_temp[q]
                                
                                if (
                                    ib_neighbor.position[0] >= ib.position[0] + ib.get_dimension()[0] and 
                                    ib_neighbor.position[1] + ib_neighbor.get_dimension()[1] > ib.position[1] and 
                                    ib_neighbor.position[2] + ib_neighbor.get_dimension()[2] > ib.position[2] + ib.get_dimension()[2] 
                                ):
                                    right_neighbor = True
                                    x_distance = ib_neighbor.position[0] - ib.position[0]
                                    ib_neigh_x_axis.append(x_distance)
                                
                                elif (
                                    ib_neighbor.position[1] > ib.position[1] + ib.get_dimension()[1] and 
                                    ib_neighbor.position[0] + ib_neighbor.get_dimension()[0] > ib.position[0] and 
                                    ib_neighbor.position[2] + ib_neighbor.get_dimension()[2] > ib.position[2] + ib.get_dimension()[2] 
                                ):
                                    front_neighbor = True
                                    y_distance = ib_neighbor.position[1] - (ib.position[1] + ib.get_dimension()[1])
                                    ib_neigh_y_axis.append(y_distance)
                                
                                elif (
                                    ib_neighbor.position[2] >= ib.position[2] + ib.get_dimension()[2] and 
                                    ib_neighbor.position[1] + ib_neighbor.get_dimension()[1] > ib.position[1] and 
                                    ib_neighbor.position[0] + ib_neighbor.get_dimension()[0] > ib.position[0] 
                                ):
                                    above_neighbor = True
                                    z_distance = ib_neighbor.position[2] - ib.position[2]
                                    ib_neigh_z_axis.append(z_distance)
                                
                                q += 1
                                
                        if not right_neighbor: 
                            x_distance = bin.length - ib.position[0]
                            ib_neigh_x_axis.append(x_distance)
                        
                        if not front_neighbor: 
                            y_distance = bin.width - ib.position[1]
                            ib_neigh_y_axis.append(y_distance)
                        
                        if not above_neighbor: 
                            z_distance = bin.height - (ib.position[2] + ib.get_dimension()[2])
                            ib_neigh_z_axis.append(z_distance)
                        
                        distance_3D = [min(ib_neigh_x_axis), min(ib_neigh_y_axis), min(ib_neigh_z_axis)]
                        pivot_dict[tuple(pivot)] = distance_3D
                
                n += 1
        
        return pivot_dict
    
    def pivot_list(self, bin, item):
        """Obtain all optional pivot points that one item could be placed into a certain bin.
        Args:
            bin: a bin in bin list that a certain item will be placed into.
            item: an unplaced item in item list.
        Returns:
            a pivot_list containing all optional pivot points that the item could be placed into a certain bin.
        """
        
        pivot_list = [] 
        
        for axis in range(0, 3): 
            items_in_bin = bin.items 
            
            for ib in items_in_bin: 
                pivot = [0, 0, 0] 
                if axis == Axis.LENGTH: # axis = 0/ x-axis
                    pivot = [ib.position[0] + ib.get_dimension()[0],
                            ib.position[1],
                            ib.position[2]]
                elif axis == Axis.WIDTH: # axis = 1/ y-axis
                    pivot = [ib.position[0],
                            ib.position[1] + ib.get_dimension()[1],
                            ib.position[2]]
                elif axis == Axis.HEIGHT: # axis = 2/ z-axis
                    pivot = [ib.position[0],
                            ib.position[1],
                            ib.position[2] + ib.get_dimension()[2]]
        
                pivot_list.append(pivot)
            
        return pivot_list 
    
    def choose_pivot_point(self, bin, item):
        """Choose the optimal one from all optional pivot points of the item after comparison.
        Args:
            bin: a bin in bin list that a certain item will be placed into.
            item: an unplaced item in item list.
        Returns:
            the optimal pivot point that a item could be placed into a bin.
        """
        
        can_put = False
        pivot_available = []
        pivot_available_temp = []
        vertex_3d = []
        vertex_2d = []
        vertex_1d = []
        
        n = 0
        m = 0
        p = 0
        
        pivot_list = self.pivot_list(bin, item)
        
        for pivot in pivot_list:
            try_put_item = bin.can_hold_item_with_rotation(item, pivot)
            
            if try_put_item:
                can_put = True
                pivot_available.append(pivot)
                pivot_temp = sorted(pivot)
                pivot_available_temp.append(pivot_temp)
        
        if pivot_available:
            while p < len(pivot_available_temp):
                vertex_3d.append(pivot_available_temp[p][0])
                p += 1
            
            p = 0
            while p < len(pivot_available_temp): 
                if pivot_available_temp[p][0] == min(vertex_3d):
                    n += 1
                    vertex_2d.append(pivot_available_temp[p][1])
                
                p += 1
        
            if n == 1:
                p = 0
                while p < len(pivot_available_temp):
                    if pivot_available_temp[p][0] == min(pivot_available_temp[p]):
                        return pivot_available[p]
                
                    p += 1
        
            else:
                p = 0
                while p < len(pivot_available_temp):
                    if (
                        pivot_available_temp[p][0] == min(pivot_available_temp[p]) and 
                        pivot_available_temp[p][1] == min(vertex_2d)
                    ):
                        m += 1
                        vertex_1d.append(pivot_available_temp[p][2])
                
                    p += 1
        
            if m == 1:
                p = 0
                while p < len(pivot_available_temp):
                    if (
                        pivot_available_temp[p][0] == min(pivot_available_temp[p]) and 
                        pivot_available_temp[p][1] == min(vertex_2d)
                    ):
                        return pivot_available[p]
                
                    p += 1
        
            else:
                p = 0
                while p < len(pivot_available_temp):
                    if (
                        pivot_available_temp[p][0] == min(pivot_available_temp[p]) and
                        pivot_available_temp[p][1] == min(vertex_2d) and
                        pivot_available_temp[p][2] == min(vertex_1d)
                    ):
                        return pivot_available[p]
                
                    p += 1
        
        if not pivot_available:
            return can_put
        
    def pack_to_bin(self, bin, item): 
        """For each item and each bin, perform whole pack process with optimal orientation and pivot point.
        Args:
            bin: a bin in bin list that a certain item will be placed into.
            item: an unplaced item in item list.
        Returns: return value is void.
        """
        
        if not bin.items:
            response = bin.put_item(item, START_POSITION, [bin.length, bin.width, bin.height])
            
            if not response:
                bin.unfitted_items.append(item)
            
            return 
        
        else:
            pivot_point = self.choose_pivot_point(bin, item)
            pivot_dict = self.pivot_dict(bin, item)
                
            if not pivot_point:
                bin.unfitted_items.append(item)
                return 
                
            distance_3D = pivot_dict[tuple(pivot_point)]
            response = bin.put_item(item, pivot_point, distance_3D)
            return  
            
    def pack(
        self, bigger_first=True, number_of_decimals=DEFAULT_NUMBER_OF_DECIMALS):
        """For a list of items and a list of bins, perform the whole pack process.
        Args:
            bin: a bin in bin list that a certain item will be placed into.
            item: an unplaced item in item list.
        Returns:
            For each bin, print detailed information about placed and unplaced items.
            Then, print the optimal bin with highest packing rate.
        """
        
        for bin in self.bins:
            bin.format_numbers(number_of_decimals)
            
        for unplaced_item in self.unplaced_items:
            unplaced_item.format_numbers(number_of_decimals)
        
        self.bins.sort(
            key = lambda bin: bin.get_volume()) # default order of bins: from smallest to biggest
        self.unplaced_items.sort(
            key = lambda unplaced_item: unplaced_item.get_volume(), reverse=bigger_first) # default order of items: from biggest to smallest
        
        filling_ratio_list = []
        
        for bin in self.bins: 
            for unplaced_item in self.unplaced_items: 
                bin.unplaced_items.append(unplaced_item) 
        
        for bin in self.bins:
            for unplaced_item in self.unplaced_items:
                self.pack_to_bin(bin, unplaced_item)
                
            print("\n:::::::::::", bin.string())
            print("FITTED ITEMS:")
            for item in bin.items:
                print("====> ", item.string())
            
            print("\nUNFITTED ITEMS:")
            for item in bin.unfitted_items:
                print("====> ", item.string())
            
            filling_ratio_list.append(bin.get_filling_ratio())
            
        max_filling_ratio = max(filling_ratio_list) 
        
        for bin in self.bins:
            if bin.get_filling_ratio() == max_filling_ratio: 
                for item in bin.items:
                    self.placed_items.append(item)
                print("\nSelected bin with highest filling ratio: ", bin.string())
                


In [32]:
import numpy as np
import time
import random
import warnings
import copy
import pandas as pd
import time
from datetime import datetime


In [53]:
###Funciones para Meta-Heuristica PSO

def func_objetivo(x):
    f=0
    Pack()
    array_item_dim = PackItem(array_item_dim) 
    f=np.matmul(array_item_dim, x.transpose()).transpose()
    return f


def activation(x, function=1):
    if (function == 1):  # Sigmoid
        return 1 / (1 + np.exp(-x))
    elif (function == 2):  # Softmax
        expo = np.exp(x)
        expo_sum = np.sum(np.exp(x))
        return expo / expo_sum
    elif (function == 3):  # ReLu
        return np.maximum(0, x)


def toDiscrete(x):
    return np.where(x < 0.5, 0, 1)

def convert_to_discrete(x):
    x_sigmoid = 1 / (1 + np.exp(-x))
    discrete_position = np.where(x_sigmoid < 0.5, 0, 1)
    return discrete_position

def calculate_velocity(w, particles_velocity, c1, c2, r1, r2, best_particle_position,
                       particles_position, best_global_position):
    inertia = w * particles_velocity
    best_particle_pos_component = r1 * (best_particle_position - particles_position)
    best_global_pos_component = r2 * (best_global_position - particles_position)

    new_velocity = inertia + c1 * best_particle_pos_component + c2 * best_global_pos_component
    return new_velocity

def calculate_best_position(objective_values, best_particle_cost, particles_position, best_particle_position, particles,
                            dimensions):
    bests = np.less(objective_values, best_particle_cost)
    reshape = np.reshape(bests, np.array([particles, 1]))
    bests_reshape = np.broadcast_to(reshape, np.array([particles, dimensions]))
    pos = np.where(bests_reshape, particles_position, best_particle_position)
    return pos

def runDiscretePSO_np(user_options, algorithm_options):
    particles = algorithm_options['particles']
    dimensions = algorithm_options['dimensions']
    objective = algorithm_options['objective']
    # For each particle, initialize position and velocity
    particles_position = np.random.uniform(-1, 1, (particles, dimensions))
    particles_velocity = np.random.uniform(-1, 1, (particles, dimensions))

    particles_position = toDiscrete(activation(particles_velocity))

    best_global = None  # Best swarm cost
    best_global_position = np.empty((particles, dimensions))  # Best swarm position
    best_particle_position = particles_position
    best_particle_cost = objective(best_particle_position)

    for k in range(0, algorithm_options['iterations']):
        objective_values = objective(best_particle_position)
        best_index = np.argmin(objective_values)
        best_value = objective_values[best_index]

        # particles x dimensions
        best_particle_position = calculate_best_position(objective_values, best_particle_cost, particles_position,
                                                         best_particle_position, particles, dimensions)

        if best_global is None or best_value < best_global:
            # Update best swarm cost and position
            best_global = best_value
            best_global_position = particles_position[best_index]

        r1 = np.random.uniform(0, 1, (particles, dimensions))
        r2 = np.random.uniform(0, 1, (particles, dimensions))

        particles_velocity = calculate_velocity(user_options['w'], particles_velocity, user_options['c1'],
                                                user_options['c2'], r1, r2, best_particle_position,
                                                particles_position, best_global_position)

        particles_position = toDiscrete(activation(particles_position + particles_velocity))

        best_particle_position = particles_position
    return best_global, best_global_position


def verify_solution(solution):
    if solution <= VolumenBin:
        return True
    return False

def print_solution(solX):
    solPru = np.random.randint(2, size=(1, n_items))
    for k in range(0, n_items):
        solPru[0, k] = solX[k]

    return solX
    #print(solPru)



def ps_bhi_np_solution(n_particles, iterations, dim, options):
    global solX_NumPy
    user_options = {'c1': options['c1'], 'c2': options['c2'], 'w': options['w']}
    algorithm_options = {'particles': n_particles, 'dimensions': dim,
                         'iterations': iterations, 'objective': func_objetivo}

    # Perform optimization
    start = time.time()
    solFO, solX_NumPy = runDiscretePSO_np(user_options, algorithm_options)
    wall_time = time.time() - start

    return solFO, wall_time



In [56]:
##FuncionPSO

def PSO():
    global array_item_dim

    average_iters = 50

    wall_times = [0, 0, 0]
    solFOs = [0, 0, 0]
    # Item 0: Pyswarms Solution
    # Item 1: PSO-BHI Solution using Numpy
    # Item 2: PSO-BHI Solution using JAX
    solutions_Total=[]
    solutions_Final=[]
    for i in range(average_iters):
        
        [solFO, wall_time] = ps_bhi_np_solution(n_particles, iteracions, dimensions, 
                        options={'c1': 0.5, 'c2': 0.3, 'w': 0.9, 'k': n_particles, 'p': 1})
        wall_times[1] += wall_time
        #solFOs[1] +=solFO
        solutions_Total.append(solFO)
        if (verify_solution(solFO)):
            solutions_Final.append(solFO)
        
        #Solution en cada Iteracion
        print("\nIteraciones Solucion ")
        print(solFO, i)
    print(len(solutions_Final))
    #print(len(solutions_Total))
        

    print("\nPSO-Binario - Time Execution: {:0.2f}ms (using NumPy)".format(1000 * wall_times[1] / average_iters))

    print("\nPSO-Binario - Solution: {:0.2f} (using NumPy)".format(solutions_Final[np.argmax(solutions_Final)]))

    print("\nPSO-Binario - Posicion (using NumPy):", print_solution(solX_NumPy))
    


In [59]:
# Length, Width, Height
L = 100.0
W = 100.0
H = 100.0

VolumenBin = L*W*H

PesoBin = 70.0

### Creamos los items basados en el paper

#Clase 1:
clase1_L = [1 , L/2.0 ]
clase1_W = [2*W/3.0 , W]
clase1_H = [2*H/3.0 , H]

#Clase 2:
clase2_L = [2*L/3.0 , L]
clase2_W = [1, W/2.0]
clase2_H = [2*H/3.0 , H]

#Clase 3:
clase3_L = [2*L/3.0 , L]
clase3_W = [2*W/3.0 , W]
clase3_H = [1 , H/2.0 ]

#Clase 4:
clase4_L = [2*L/3.0 , L]
clase4_W = [2*W/3.0 , W]
clase4_H = [1 , H/2.0 ]

#Clase 5:
clase5_L = [2*L/3.0 , L]
clase5_W = [2*W/3.0 , W]
clase5_H = [1 , H/2.0 ]


num_items = 10
array_items = []
array_item_dim = []

for i in np.arange(num_items):
    #posicion[i] = random.uniform(limites_inf[i],self.limites_sup[i])
    x = random.uniform(clase1_L[0],clase1_L[1])
    y = random.uniform(clase1_W[0],clase1_W[1])
    z = random.uniform(clase1_H[0],clase1_H[1])
    array_items.append([x,y,z])
    array_item_dim.append(x*y*z)
    

    
n_items = 10
dimensions = n_items
iteracions = 100
n_particles = 100

PSO()


Iteraciones Solucion 
276226.38532175194 0

Iteraciones Solucion 
151852.71999035106 1

Iteraciones Solucion 
319113.1593836212 2

Iteraciones Solucion 
192916.6459520397 3

Iteraciones Solucion 
317290.31128344056 4

Iteraciones Solucion 
151852.71999035106 5

Iteraciones Solucion 
204306.30165211667 6

Iteraciones Solucion 
192916.6459520397 7

Iteraciones Solucion 
86360.59412937902 8

Iteraciones Solucion 
106556.05182266068 9

Iteraciones Solucion 
167260.43939327012 10

Iteraciones Solucion 
151852.71999035106 11

Iteraciones Solucion 
269798.4275130887 12

Iteraciones Solucion 
167260.43939327012 13

Iteraciones Solucion 
408046.72664615326 14

Iteraciones Solucion 
106556.05182266068 15

Iteraciones Solucion 
297094.85359015886 16

Iteraciones Solucion 
167260.43939327012 17

Iteraciones Solucion 
290666.8957814957 18

Iteraciones Solucion 
192916.6459520397 19

Iteraciones Solucion 
269798.4275130887 20

Iteraciones Solucion 
319113.1593836212 21

Iteraciones Solucion 
167260

In [60]:

num_items = 20
array_items = []
array_item_dim = []

for i in np.arange(num_items):
    #posicion[i] = random.uniform(limites_inf[i],self.limites_sup[i])
    x = random.uniform(clase1_L[0],clase1_L[1])
    y = random.uniform(clase1_W[0],clase1_W[1])
    z = random.uniform(clase1_H[0],clase1_H[1])
    array_items.append([x,y,z])
    array_item_dim.append(x*y*z)
    
n_items = 20
dimensions = n_items
iteracions = 100
n_particles = 100

PSO()


Iteraciones Solucion 
826972.7924058711 0

Iteraciones Solucion 
865296.5917981355 1

Iteraciones Solucion 
798850.8406616903 2

Iteraciones Solucion 
485988.17423763225 3

Iteraciones Solucion 
931861.8411897204 4

Iteraciones Solucion 
682484.7846757473 5

Iteraciones Solucion 
697280.7571396591 6

Iteraciones Solucion 
705285.6819642973 7

Iteraciones Solucion 
811553.3431644985 8

Iteraciones Solucion 
666157.8326346884 9

Iteraciones Solucion 
792744.365919936 10

Iteraciones Solucion 
713326.6382987972 11

Iteraciones Solucion 
955922.9185180293 12

Iteraciones Solucion 
704727.0160461128 13

Iteraciones Solucion 
634526.991333778 14

Iteraciones Solucion 
584997.5353070474 15

Iteraciones Solucion 
653304.9777888946 16

Iteraciones Solucion 
870631.1306895143 17

Iteraciones Solucion 
34981.8719381625 18

Iteraciones Solucion 
988268.3262791301 19

Iteraciones Solucion 
1047669.8099795526 20

Iteraciones Solucion 
899107.2699332165 21

Iteraciones Solucion 
1125256.470317539 22

In [63]:

num_items = 25
array_items = []
array_item_dim = []

for i in np.arange(num_items):
    #posicion[i] = random.uniform(limites_inf[i],self.limites_sup[i])
    x = random.uniform(clase1_L[0],clase1_L[1])
    y = random.uniform(clase1_W[0],clase1_W[1])
    z = random.uniform(clase1_H[0],clase1_H[1])
    array_items.append([x,y,z])
    array_item_dim.append(x*y*z)
    
n_items = 25
dimensions = n_items
iteracions = 100
n_particles = 100

PSO()


Iteraciones Solucion 
653374.7903515596 0

Iteraciones Solucion 
1309848.6935886415 1

Iteraciones Solucion 
1252966.3402528882 2

Iteraciones Solucion 
1161209.6871430431 3

Iteraciones Solucion 
1067182.83862656 4

Iteraciones Solucion 
1381393.2902928432 5

Iteraciones Solucion 
1130518.677256674 6

Iteraciones Solucion 
1300619.053722711 7

Iteraciones Solucion 
913295.2015562278 8

Iteraciones Solucion 
1087106.8234788387 9

Iteraciones Solucion 
1101766.3606201229 10

Iteraciones Solucion 
1020670.0062456826 11

Iteraciones Solucion 
982338.680438572 12

Iteraciones Solucion 
887858.0880543791 13

Iteraciones Solucion 
1047909.8404707965 14

Iteraciones Solucion 
1037998.0513431855 15

Iteraciones Solucion 
1083826.7967000287 16

Iteraciones Solucion 
1212744.1201655252 17

Iteraciones Solucion 
1038927.3231324635 18

Iteraciones Solucion 
998470.6736665074 19

Iteraciones Solucion 
1047936.0463585871 20

Iteraciones Solucion 
1315940.2044511877 21

Iteraciones Solucion 
805957.

In [65]:

num_items = 35
array_items = []
array_item_dim = []

for i in np.arange(num_items):
    #posicion[i] = random.uniform(limites_inf[i],self.limites_sup[i])
    x = random.uniform(clase1_L[0],clase1_L[1])
    y = random.uniform(clase1_W[0],clase1_W[1])
    z = random.uniform(clase1_H[0],clase1_H[1])
    array_items.append([x,y,z])
    array_item_dim.append(x*y*z)
    
n_items = 35
dimensions = n_items
iteracions = 100
n_particles = 100

PSO()


Iteraciones Solucion 
1176806.5636229755 0

Iteraciones Solucion 
1923080.0688766986 1

Iteraciones Solucion 
1759074.9676125497 2

Iteraciones Solucion 
1230110.937496679 3

Iteraciones Solucion 
1126063.2072258608 4

Iteraciones Solucion 
1846385.8476049774 5

Iteraciones Solucion 
1718501.2567643293 6

Iteraciones Solucion 
1270441.137527256 7

Iteraciones Solucion 
1731792.1153756673 8

Iteraciones Solucion 
1982985.9921704947 9

Iteraciones Solucion 
1778620.4997712034 10

Iteraciones Solucion 
1510732.705011144 11

Iteraciones Solucion 
1842274.98176675 12

Iteraciones Solucion 
1333439.0632603623 13

Iteraciones Solucion 
1569415.4587198573 14

Iteraciones Solucion 
1579056.1229520424 15

Iteraciones Solucion 
1509162.7225401383 16

Iteraciones Solucion 
1775711.836604189 17

Iteraciones Solucion 
1317643.5901465174 18

Iteraciones Solucion 
2054933.4953741292 19

Iteraciones Solucion 
1207714.5456402202 20

Iteraciones Solucion 
1693116.7300343104 21

Iteraciones Solucion 
157

In [66]:
# 2. GRASP Meta-Heuristic

In [226]:
# Length, Width, Height
L = 100.0
W = 100.0
H = 100.0

VolumenBin = L*W*H

PesoBin = 70.0

### Creamos los items basados en el paper

#Clase 1:
clase1_L = [1 , L/2.0 ]
clase1_W = [2*W/3.0 , W]
clase1_H = [2*H/3.0 , H]

#Clase 2:
clase2_L = [2*L/3.0 , L]
clase2_W = [1, W/2.0]
clase2_H = [2*H/3.0 , H]

#Clase 3:
clase3_L = [2*L/3.0 , L]
clase3_W = [2*W/3.0 , W]
clase3_H = [1 , H/2.0 ]

#Clase 4:
clase4_L = [2*L/3.0 , L]
clase4_W = [2*W/3.0 , W]
clase4_H = [1 , H/2.0 ]

#Clase 5:
clase5_L = [2*L/3.0 , L]
clase5_W = [2*W/3.0 , W]
clase5_H = [1 , H/2.0 ]


num_items = 10
array_items = []
array_item_dim = []

for i in np.arange(num_items):
    #posicion[i] = random.uniform(limites_inf[i],self.limites_sup[i])
    x = random.uniform(clase1_L[0],clase1_L[1])
    y = random.uniform(clase1_W[0],clase1_W[1])
    z = random.uniform(clase1_H[0],clase1_H[1])
    array_items.append([x,y,z])
    array_item_dim.append(x*y*z)

#Read and Write the file
with open('input20.in', 'a') as fd:
    i=0
    fd.writelines(str(num_items))
    fd.writelines("\n")
    
    for d in array_item_dim:
        i+=1
        e=np.random.randint(10, 200)
        fd.writelines(str(i)+" "+str(e)+" "+str(d))
        fd.writelines("\n")
    fd.writelines(str(VolumenBin))
    fd.writelines("\n")

#Open the file
infile = open('input20.in', 'r')
data = infile.read()
print(data)

infile.close()

10
1 97 25394.496462985204
2 73 224458.03153005653
3 135 33671.14161366579
4 22 200730.27992332095
5 33 95686.16139715603
6 23 251868.87404147306
7 141 208578.09596919804
8 156 260937.78329497206
9 92 269655.9582471943
10 29 215740.18673036536
1000000.0



In [215]:
import random
import re
import numpy as np


def cost(solution, weight, value, id, capacity):
    max_value = 0
    for j in range(len(solution)):
        if solution[j] == 1:
            max_value += value[j]

    return max_value


def local_search(solution, weight, value, id, capacity):
    temp = solution[:]
    # print(len(value))
    for i in range(len(temp)):
        max_temp_weight = 0
        max_value_temp = 0
        max_value = 0
        if temp[i] == 0:
            temp[i] = 1
            for j in range(len(temp)):
                if temp[j] == 1:
                    max_value_temp += value[id[j]]
                    max_temp_weight += weight[id[j]]
            temp[i] = 0
        elif temp[i] == 1:
            temp[i] = 0
            max_value_temp = 0
            max_value = 0
            for j in range(len(temp)):
                if temp[j] == 1:
                    max_value_temp += value[id[j]]
                    max_temp_weight += weight[id[j]]

            temp[i] = 1
        for j in range(len(solution)):
            if solution[j] == 1:
                max_value += value[id[j]]
        if capacity - max_temp_weight >= 0 and max_value_temp > max_value:
            solution = temp[:]

    return solution


def Greedy_Randomized_Construction(id, value, weight, capacity):
    solucao = []
    item = {}
    for i in range(len(id)):
        item[i] = np.divide(value[i], weight[i]), value[i], weight[i]
    item = sorted(item.values(), reverse=True)

    item_temp = item[:]
    capacidade_restante = capacity

    peso = 0

    while len(item) > 0:
        lcr = []
        for i in range(2):
            if i < len(item):
                lcr.append(item[i])

        s = random.choice(lcr)
        if s[2] <= capacidade_restante:
            solucao.append(s[1])
            capacidade_restante -= s[2]
            peso += s[2]

        item.pop(item.index(s))

    sol = [0 for i in range(len(value))]

    for item in value:
        if item in solucao:
            sol[value.index(item)] = 1

    return sol, peso


def grasp(max_iterations, id, value, weight, capacity):
    start = time.time()
    best = 0
    for k in range(max_iterations):
        id_temp = []
        for i in range(len(id)):
            id_temp.append(id[i])
        solution, peso = Greedy_Randomized_Construction(id_temp, value, weight, capacity)

        solution = local_search(solution, weight, value, id, capacity)
        solution = cost(solution, weight, value, id, capacity)
        if solution > best:
            best = solution
    wall_time = time.time() - start
    return best, peso, solution, wall_time



In [227]:
def grasp_runtime():
    iterator = 20
    while iterator <= 20:
        #file = open("entradas/input" + str(iterator) + ".in", "r")
        file = open("input" + str(iterator) + ".in", "r")

        iterator += 1
        weight = []
        value = []
        id = []
        i = 0
        n = int()
        capacity = int()
        for line in file:
            i += 1
            line = line.rstrip()
            numbers = re.findall("[0-9]+", line)
            if i == 1:
                n = int(numbers[0])
            elif 1 < i <= n + 1:
                id.append(int(numbers[0]) - 1)
                value.append(int(numbers[1]))
                weight.append(int(numbers[2]))
            else:
                capacity = int(numbers[0])

        max_iterations = 100
        s = "Instancia " + str(iterator - 1) + " : " + str(grasp(max_iterations, id, value, weight, capacity)) + "\n"
        print(s)
        #file = open("Output/grasp.out", "a+")
        #file = open("grasp.out", "a+")
        #file.write(s)



In [228]:
grasp_runtime()


Instancia 20 : (654, 893921, 654, 0.007393360137939453)

